**CNNによる2次元顔画像表情判定**

In [2]:
!ps aux
!kill -9 <pid>

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.2  0.0    992     4 ?        Ss   11:22   0:00 /sbin/docker-
root           7  0.2  0.3 337512 47208 ?        Sl   11:22   0:00 /tools/node/b
root          17  0.2  0.0  35892  4856 ?        Ss   11:22   0:00 tail -n +0 -F
root          40  0.2  0.3 160352 41908 ?        S    11:22   0:00 python3 /usr/
root          65  0.6  0.4 193896 60128 ?        Sl   11:22   0:01 /usr/bin/pyth
root          66  0.0  0.0 706568  5536 ?        Sl   11:22   0:00 /usr/local/bi
root          77 10.6  0.8 699196 115424 ?       Ssl  11:25   0:01 /usr/bin/pyth
root          97  0.2  0.1 126360 14444 ?        Sl   11:26   0:00 /usr/bin/pyth
root         118  0.0  0.0  59036  6408 ?        R    11:26   0:00 ps aux
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `kill -9 <pid>'


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten,MaxPooling2D,Conv2D
from keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

**訓練画像、検証画像、テスト画像のディレクトリ**

In [5]:
classes = ['angry', 'disgust','fear','happy','neutral','sad','suprise']
nb_classes = len(classes)
batch_size_for_data_generator = 20

base_dir = '/content/drive/MyDrive/kaggle/Facial Experience'

train_dir = base_dir+'/train'
validation_dir = base_dir+'/valid'
test_dir = base_dir+'/test'

img_rows, img_cols = 48, 48

os.path.join(base_dir,'path')

**ImageDataGeneratorを使って画像データを拡張する**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(img_rows, img_cols),
                                                    color_mode='rgb',
                                                    classes=classes,
                                                    class_mode='categorical',
                                                    batch_size=1117,
                                                    shuffle=True)

Found 84892 images belonging to 7 classes.


train 84892枚
2x2x19x1117
バッジサイズ2048

In [ ]:
validation_generator = test_datagen.flow_from_directory(directory=validation_dir,
                                                        target_size=(img_rows, img_cols),
                                                        color_mode='rgb',
                                                        classes=classes,
                                                        class_mode='categorical',
                                                        batch_size=131,
                                                        shuffle=True)

Found 21615 images belonging to 7 classes.


valid 21615枚 3x5x11x131 バッジサイズ2048

**CNNモデル**

なんで512?→特に理由なし1024でもなんでもいい

dropoutの役割→層から層への伝達の際に出力データを意図的に0とする事で、訓練データに過剰適合する問題を回避する。

出力層
二値分類→sigmoid
他クラス分類→softmax
回帰分類→恒等関数

モデル参照先
https://sciresol.s3.us-east-2.amazonaws.com/IJST/Articles/2019/Issue-24/Article9.pdf

leaky relu

relu関数においてx<0のとき正の傾きを与える

In [ ]:
model=Sequential()
# 畳み込み層
model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(img_rows, img_cols, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.2))

# 全結合層
model.add(Flatten())
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.2))

# 出力層
model.add(Dense(nb_classes, activation='softmax'))
          
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         7

In [ ]:
opt = Adam(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer= opt, metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


**コールバックを使った学習**

In [8]:
hdf5_file = os.path.join(base_dir, 'model.hdf5')

再度ここから

In [ ]:
from keras.callbacks import ModelCheckpoint
modelCheckpoint = ModelCheckpoint(filepath = hdf5_file,
                                  monitor='val_loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=False,
                                  mode='min',
                                  period=1)

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=76,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=165,
                    callbacks=[modelCheckpoint],
                    verbose=1)

Epoch 1/5
17/76 [=====>........................] - ETA: 3:25:36 - loss: 1.6143 - acc: 0.3263

In [ ]:
model.save(hdf5_file)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

**テスト**

In [6]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [9]:
model=keras.models.load_model(hdf5_file)

In [11]:
import numpy as np
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=(img_rows, img_cols),
                                                  color_mode='rgb',
                                                  classes=classes,
                                                  class_mode='categorical',
                                                  batch_size=32,
                                                  shuffle=False)

test_steps_per_Epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)

Found 16021 images belonging to 7 classes.


In [14]:
predictions = model.predict_generator(test_generator, steps=test_steps_per_Epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)
# Ground-Truthクラスとクラスラベルを取得する
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


正解率（accuracy）: accuracy_score()

適合率（precision）: precision_score()

再現率（recall）: recall_score()

F1値（F1-measure）: f1_score()

In [17]:
from sklearn import metrics
# scikit-learnを使用して統計を取得する
report = metrics.classification_report(true_classes, 
                                       predicted_classes, 
                                       labels=np.arange(len(classes)),
                                       target_names=classes)
print(report)

              precision    recall  f1-score   support

       angry       0.50      0.60      0.55      1041
     disgust       0.36      0.01      0.01       655
        fear       0.50      0.34      0.41       691
       happy       0.86      0.45      0.59      5469
     neutral       0.55      0.85      0.67      5988
         sad       0.39      0.38      0.39      2177
     suprise       0.00      0.00      0.00         0

   micro avg       0.58      0.58      0.58     16021
   macro avg       0.45      0.38      0.37     16021
weighted avg       0.62      0.58      0.56     16021



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**参考**


*   [Keras / CNN] 多クラス画像分類 --- ラーメンの味分類

    https://qiita.com/Phoeboooo/items/cfe8560fe8a285855340
*   リスト項目

